# **Семинар 2 — Версионирование данных и моделей с помощью DVC и Git**

## **Цель семинара**

На этом семинаре вы познакомитесь с инструментом **DVC (Data Version Control)** — системой для версионирования данных и моделей в ML-проектах.

Вы научитесь:
- Версионировать большие файлы (датасеты, модели) вместе с кодом.
- Настраивать удалённое хранилище для данных.
- Сохранять и восстанавливать версии моделей.
- Создавать пайплайны для автоматического воспроизводства экспериментов.

---

## **1. Введение**

Git отлично справляется с версиями кода, но в ML-проектах у нас есть ещё:
- большие датасеты (гигабайты, терабайты);
- бинарные модели (.pkl, .h5, .pt);
- пайплайны обработки данных.

Git не подходит для таких файлов — он не умеет эффективно хранить и сравнивать большие бинарные данные.

Поэтому используется **DVC** — инструмент для версионирования данных и моделей поверх Git.  
Он позволяет:
- хранить только *метаданные* файлов в Git;
- синхронизировать сами данные в отдельном хранилище (локальном или облачном);
- воспроизводить эксперименты и пайплайны.


In [ ]:
!pip install dvc -q

## **2. Инициализация проекта (Git + DVC)**

В реальном проекте вы бы клонировали репозиторий.  
Для целей семинара создадим пустой Git-репозиторий прямо в среде и инициализируем DVC.

Шаги:
1. Инициализировать Git и настроить имя/почту (локально для этой папки).
2. Инициализировать DVC.
3. Зафиксировать первый коммит.


In [ ]:
# создадим рабочую папку проекта (по желанию можно пропустить и работать в корне)
import os, shutil, subprocess, textwrap, sys, json, pathlib

project_dir = "mlops_dvc_demo"
if os.path.exists(project_dir):
    shutil.rmtree(project_dir)
os.makedirs(project_dir, exist_ok=True)
%cd $project_dir

# инициализация Git
!git init -q
!git config user.name "student"
!git config user.email "student@example.com"

# инициализация DVC
!dvc init -q

# базовые служебные файлы
open(".gitignore", "a").close()
open(".dvcignore", "a").close()

# первый коммит
!git add .
!git commit -m "Init Git + DVC"
!git log --oneline -n 1


## **3. Добавление датасета в DVC**

Сымитируем наличие датасета: сформируем `data/train.csv` на основе Iris.  
Затем:
- добавим файл в DVC (`dvc add`),
- проверим, что появился мета-файл `train.csv.dvc` и запись в `.gitignore`,
- зафиксируем изменения в Git.

Важно: **сам CSV не хранится в Git**, в репозиторий попадут только его метаданные.


In [ ]:
# создадим папку и сгенерируем небольшой CSV (Iris)
import pandas as pd
from sklearn import datasets
import os

os.makedirs("data", exist_ok=True)
iris = datasets.load_iris(as_frame=True)
df = iris.frame.rename(columns={"target": "label"})
df.to_csv("data/train.csv", index=False)

# добавим в DVC
!dvc add data/train.csv -q

# посмотрим, что появилось
!ls -la data
print("\nСодержимое .gitignore:")
print(open(".gitignore").read())

# зафиксируем в Git метаданные датасета
!git add data/.gitignore data/train.csv.dvc
!git commit -m "Add dataset v1 via DVC"
!git status -s


## **4. Подключение remote-хранилища и отправка данных**

DVC хранит сами файлы (большие данные/модели) в удалённом хранилище.  
Для демонстрации используем **локальную папку** как remote.

Шаги:
1. Добавить remote `localstorage` и сделать его значением по умолчанию (`-d`).
2. Отправить данные в хранилище (`dvc push`).
3. Убедиться, что данные ушли в папку `./dvc-storage`.


In [ ]:
# добавим локальный remote
!dvc remote add -d localstorage ./dvc-storage
!git add .dvc/config
!git commit -m "Configure DVC remote storage (local)"

# отправим данные
!dvc push -q

# проверим, что в dvc-storage появились объекты
!ls -R dvc-storage | head -n 50


## **5. Имитация воспроизведения у коллеги (pull)**

Проверим типичный сценарий:
- «коллега» клонирует репозиторий,
- выполняет `dvc pull`,
- получает ту же версию датасета.

Мы смоделируем это, удалив локальные данные и восстановив их из хранилища.


In [ ]:
# удалим локальный файл данных, оставив только метаданные .dvc
import os, shutil
if os.path.exists("data/train.csv"):
    os.remove("data/train.csv")

print("До pull, есть ли data/train.csv? ->", os.path.exists("data/train.csv"))

# восстановим данными из remote
!dvc pull -q

print("После pull, есть ли data/train.csv? ->", os.path.exists("data/train.csv"))
!wc -l data/train.csv


## **6. Версионирование модели через DVC**

Обучим простую модель (LogisticRegression) из `data/train.csv`,  
сохраним её как `models/model.pkl` и добавим в DVC.

Шаги:
1. Написать короткий `train.py` (читает CSV, обучает, сохраняет модель).
2. Запустить `python train.py`.
3. Добавить `models/model.pkl` в DVC и отправить в remote (`dvc push`).


In [ ]:
# создадим скрипт обучения
os.makedirs("models", exist_ok=True)
train_py = r"""
import pandas as pd
from sklearn.linear_model import LogisticRegression
import joblib, os

df = pd.read_csv("data/train.csv")
X = df.drop("label", axis=1)
y = df["label"]

model = LogisticRegression(max_iter=300)
model.fit(X, y)

os.makedirs("models", exist_ok=True)
joblib.dump(model, "models/model.pkl")
print("Saved models/model.pkl")
"""
open("train.py", "w").write(train_py)

# зависимости
!pip install scikit-learn joblib -q

# обучим модель
!python train.py

# добавим модель в DVC и запушим в remote
!dvc add models/model.pkl -q
!git add models/model.pkl.dvc models/.gitignore train.py
!git commit -m "Add model v1 via DVC"
!dvc push -q

# проверим, что файл модели можно восстановить
!rm -f models/model.pkl
print("Файл модели удалён локально:", not os.path.exists("models/model.pkl"))
!dvc pull -q
print("Файл модели восстановлён:", os.path.exists("models/model.pkl"))


## **7. Пайплайны DVC (dvc.yaml) и воспроизведение**

DVC позволяет описать связи «данные → код → артефакты» в `dvc.yaml`.  
Добавим простейший пайплайн с одной стадией `train`:

- `cmd`: как запустить обучение;
- `deps`: зависимости (скрипты, данные);
- `outs`: выходы (модель).

После этого командой `dvc repro` можно автоматически выявить,  
что нужно пересчитать (например, если изменились данные или код).


In [ ]:
dvc_yaml = """stages:
  train:
    cmd: python train.py
    deps:
      - train.py
      - data/train.csv
    outs:
      - models/model.pkl
"""
open("dvc.yaml", "w").write(dvc_yaml)

!git add dvc.yaml
!git commit -m "Add DVC pipeline (train stage)"

# проверим воспроизведение
!dvc repro -q
!echo "Пайплайн успешно воспроизведён."


## **8. Откат версий данных/модели (Git + DVC checkout)**

Типичный сценарий:
1. Меняем датасет (например, фильтруем часть строк), коммитим изменения — получаем **v2**.
2. Хотим вернуться к **v1**: делаем `git checkout <commit>` и `dvc checkout`.

Ниже мы:
- создадим **v2** датасета,
- зафиксируем его,
- затем вернёмся на предыдущий коммит и восстановим **v1**.


In [ ]:
import pandas as pd

# Сформируем версию v2: отфильтруем часть строк
df = pd.read_csv("data/train.csv")
df_v2 = df.sample(frac=0.8, random_state=123).reset_index(drop=True)
df_v2.to_csv("data/train.csv", index=False)

# Обновим версию файла в DVC
!dvc add data/train.csv -q
!git add data/train.csv.dvc data/.gitignore
!git commit -m "Dataset v2 (filtered 80%)"
!dvc push -q

print("Текущая длина данных (v2):")
!wc -l data/train.csv

# сохраним id текущего и предыдущего коммитов
last2 = !git log --pretty=format:'%H' -n 2
curr_commit = last2[0]
prev_commit = last2[1]
print("\nТекущий коммит:", curr_commit)
print("Предыдущий коммит:", prev_commit)

# переключимся на предыдущий (где была v1) и выполним dvc checkout
!git checkout -q $prev_commit
!dvc checkout -q

print("\nПосле отката к v1 длина данных:")
!wc -l data/train.csv


## **9. Обсуждение: DVC vs MLflow и интеграция**

- **MLflow** — про эксперименты: параметры, метрики, артефакты, модели, Registry.  
- **DVC** — про данные и артефакты на уровне файлов/папок и их воспроизводимость (пайплайны).

Как связать:
- использовать MLflow для логирования метрик/моделей,
- хранить сами датасеты/модели в DVC remote,
- запускать пайплайны `dvc repro` в CI/CD, перед шагом обучения/оценки.

Практические вопросы:
- Какие remote-хранилища использовать (S3/GCS/Azure/SSH/локальные)?
- Где хранить секреты (доступ к облачным бакетам)?
- Как кэшировать данные на CI, чтобы ускорить `dvc pull`/`dvc push`?


In [ ]:
!pip install dvc mlflow scikit-learn pandas numpy matplotlib

## Шаг 1. Инициализация DVC

DVC — это инструмент для:

- версионирования данных (как Git для данных),
- построения воспроизводимых ML-пайплайнов,
- автоматического трекинга метрик и артефактов.

Создадим структуру проекта:

project/

data/

src/

dvc.yaml

params.yaml

mlruns/ (для MLflow)

Инициализируем DVC:

In [1]:
!dvc init

/bin/bash: line 1: dvc: command not found


## Шаг 2. Генерация данных и их версионирование

Создадим простой набор данных (train.csv), который будем отслеживать в DVC.

In [ ]:
import pandas as pd
import numpy as np

df = pd.DataFrame({
    "x": np.linspace(-3, 3, 200),
})
df["y"] = 2 * df["x"] + np.random.normal(0, 1, size=len(df))

df.to_csv("data.csv", index=False)
df.head()


## Шаг 3. Добавляем данные под контроль DVC

Эта команда создаёт:

- `data.csv.dvc` — мета-файл
- данные переносятся в `.dvc/cache/`



In [ ]:
!dvc add data.csv


## Шаг 4. Параметры обучения (params.yaml)

DVC умеет подхватывать параметры из файла `params.yaml`.



In [ ]:
%%writefile params.yaml
train:
  test_size: 0.2
  random_state: 42
  alpha: 0.1


## Шаг 5. Скрипт обучения модели с MLflow внутри DVC stage

Это ключевой момент семинара:
**MLflow будет логировать параметры и метрики, а DVC — контролировать пайплайн.**



In [ ]:
%%writefile train.py
import mlflow
import pandas as pd
import yaml
import numpy as np
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

# Загружаем параметры
params = yaml.safe_load(open("params.yaml"))["train"]

# DVC читает данные отслеживаемые в data.csv.dvc
df = pd.read_csv("data.csv")
X = df[["x"]]
y = df["y"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=params["test_size"],
    random_state=params["random_state"]
)

model = Ridge(alpha=params["alpha"])
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)

# Логируем в MLflow
mlflow.set_tracking_uri("file://" + __import__("os").path.abspath("mlruns"))
mlflow.set_experiment("dvc_pipeline_experiment")

with mlflow.start_run():
    mlflow.log_param("alpha", params["alpha"])
    mlflow.log_metric("mse", mse)
    mlflow.sklearn.log_model(model, "model")

# Сохраняем метрики для DVC
with open("metrics.json", "w") as f:
    f.write('{"mse": %f}' % mse)


## Шаг 6. Создаём DVC пайплайн (dvc.yaml)

Здесь происходит магия:

- DVC знает, что входные данные — `data.csv`
- параметры — `params.yaml`
- скрипт запуска — `python train.py`
- выход — `metrics.json`



In [ ]:
%%writefile dvc.yaml
stages:
  train:
    cmd: python train.py
    deps:
      - train.py
      - data.csv
      - params.yaml
    metrics:
      - metrics.json:
          cache: false


## Шаг 7. Запускаем пайплайн



In [ ]:
!dvc repro


# Результат: настоящая мини-MLOps система

Теперь у нас:

### ✔ DVC
- отслеживает данные
- управляет пайплайном
- знает, что изменилось и когда нужно пересчитать

### ✔ MLflow
- логирует эксперименты
- хранит модели
- визуализирует метрики

### ✔ Связка DVC + MLflow
- полная воспроизводимость
- версионность данных
- версионность моделей
- автоматическое обучение при изменении данных или параметров

### Это и есть современный ML workflow.


# Что можно добавить на семинаре

- удалённые хранилища DVC (SSH / S3)
- GitHub + DVC + MLflow — полный CI/CD цикл
- DVC CML: автоматические эксперименты по pull-request
- Автоматический выбор победителя через MLflow + DVC metrics

Готов подготовить полный модуль, если хочешь развить курс дальше.


## **10. Чек-лист к концу семинара**

К моменту завершения семинара студент должен уметь продемонстрировать:

1. **Инициализацию проекта**  
   - Создан Git-репозиторий.  
   - Выполнен `dvc init`, первый коммит зафиксирован.

2. **Версионирование датасета**  
   - Файл `data/train.csv` добавлен в DVC (`dvc add`).  
   - Появились файлы `train.csv.dvc` и запись в `.gitignore`.  
   - Коммит с сообщением `"Add dataset v1 via DVC"` присутствует в истории.

3. **Подключение remote-хранилища**  
   - Настроено локальное хранилище `./dvc-storage`.  
   - Выполнен `dvc push`, данные успешно отправлены.

4. **Воспроизведение данных**  
   - После удаления локальных данных команда `dvc pull` восстанавливает датасет из remote.

5. **Версионирование модели**  
   - Модель обучена (скрипт `train.py`) и сохранена как `models/model.pkl`.  
   - Файл добавлен в DVC и загружен в хранилище.

6. **Пайплайн `dvc.yaml`**  
   - Создана стадия `train` с зависимостями и выходами.  
   - Команда `dvc repro` успешно воспроизводит пайплайн.

7. **Откат версий**  
   - После перехода на предыдущий коммит и `dvc checkout` восстанавливается версия `v1` датасета.

8. **Общее понимание**  
   - Студент может объяснить, чем DVC отличается от MLflow и как они дополняют друг друга.

> **Результат:** в репозитории присутствуют все артефакты (`.dvc`, `dvc.yaml`, `train.py`, `data/`, `models/`), а пайплайн полностью воспроизводим с помощью `dvc repro`.
